In [ ]:
# Fuel poverty stats for Lewisham.
# Data: Department for Business, Energy & Industrial Strategy
# URL: https://www.gov.uk/government/statistics/sub-regional-fuel-poverty-data-2021

# Loads fuel poverty data and prepares it for use in Lewisham ward profiles.
#
# This is not be available at Ward level, however we can annotate 
# which LSOAs to show for which ward.

In [ ]:
import numpy as np
import pandas as pd

from google.colab import drive
import google.colab.files as files

import IPython

Data
==
GDrive mount
--

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Used as root folder.
project_dir = '/content/gdrive/MyDrive/WardProfiles'
ward_lookups_dir = f"{project_dir}/lookups/2022_wards"

# For exports
out_dir = f"{project_dir}/fuel_poverty"

In [ ]:
!mkdir -p '{out_dir}'

Downloads
--

In [ ]:
# 2019 Fuel Poverty, at LSOA level
# Source: 
# https://www.gov.uk/government/statistics/english-indices-of-deprivation-2019
!wget 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/981910/2021-sub-regional-fuel-poverty-tables.xlsx' \
  -O "{out_dir}/2021-sub-regional-fuel-poverty-tables.xlsx"

--2021-11-05 11:39:07--  https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/981910/2021-sub-regional-fuel-poverty-tables.xlsx
Resolving assets.publishing.service.gov.uk (assets.publishing.service.gov.uk)... 151.101.0.144, 151.101.64.144, 151.101.128.144, ...
Connecting to assets.publishing.service.gov.uk (assets.publishing.service.gov.uk)|151.101.0.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2242510 (2.1M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/fuel_poverty/2021-sub-regional-fuel-poverty-tables.xlsx’

/content/gdrive/MyD 100%[===================>]   2.14M  --.-KB/s    in 0.05s   

2021-11-05 11:39:09 (41.8 MB/s) - ‘/content/gdrive/MyDrive/WardProfiles/fuel_poverty/2021-sub-regional-fuel-poverty-tables.xlsx’ saved [2242510/2242510]



In [ ]:
!ls -lh "{out_dir}"

total 2.2M
-rw------- 1 root root 2.2M Apr 28  2021 2021-sub-regional-fuel-poverty-tables.xlsx


Lookups
--
Used to match up LSOAs with 2022 electoral wards

In [ ]:
# All LSOAs and their 2022 wards in Lewisham
lsoa_wd22_join = pd.read_csv(f"{ward_lookups_dir}/lbl_lsoa11_wd22_proposed.csv")
lsoa_wd22_join = lsoa_wd22_join[['LSOA11CD', 'WD22CD_proposed']]
lsoa_wd22_join.head()

,LSOA11CD,WD22CD_proposed
0,E01003316,E05013729
1,E01003314,E05013729
2,E01003312,E05013729
3,E01003313,E05013729
4,E01003310,E05013729


Process & export
==

In [ ]:
# For filtering of source data
lbl_lsoa_list = lsoa_wd22_join.LSOA11CD.unique() # All LSOAs in Lewisham
len(lbl_lsoa_list)

169

In [ ]:
# Source data
d = pd.read_excel(f"{out_dir}/2021-sub-regional-fuel-poverty-tables.xlsx", 
                  sheet_name='Table 3',
                  skiprows=2, # Data starts in row 3
                  skipfooter=9
                  ) # Data has some footer rows
d.head()

,LSOA Code,LSOA Name,LA Code,LA Name,Region,Number of households1,Number of households in fuel poverty1,Proportion of households fuel poor (%)
0,E01000001,City of London 001A,E09000001,City of London,London,928,59,6.357759
1,E01000002,City of London 001B,E09000001,City of London,London,882,25,2.834467
2,E01000003,City of London 001C,E09000001,City of London,London,878,95,10.820046
3,E01000005,City of London 001E,E09000001,City of London,London,502,61,12.151394
4,E01000006,Barking and Dagenham 016A,E09000002,Barking and Dagenham,London,587,88,14.991482


In [ ]:
d.tail()

,LSOA Code,LSOA Name,LA Code,LA Name,Region,Number of households1,Number of households in fuel poverty1,Proportion of households fuel poor (%)
32839,E01033764,Liverpool 022E,E08000012,Liverpool,North West,1258,206,16.375199
32840,E01033765,Liverpool 061D,E08000012,Liverpool,North West,687,106,15.429403
32841,E01033766,Liverpool 042G,E08000012,Liverpool,North West,454,94,20.704846
32842,E01033767,Liverpool 050J,E08000012,Liverpool,North West,591,121,20.473773
32843,E01033768,Liverpool 037F,E08000012,Liverpool,North West,863,207,23.986095


In [ ]:
# Standardise column names
d = d.rename(columns={
    'LSOA Code': 'LSOA11CD', 
    'LSOA Name': 'LSOA11NM',
    'LA Code': 'LAD19CD',
    'LA Name': 'LAD19NM',
    'Number of households1': 'Number of households',
    'Number of households in fuel poverty1': 'Number of households in fuel poverty'
})
d.columns

Index(['LSOA11CD', 'LSOA11NM', 'LAD19CD', 'LAD19NM', 'Region',
       'Number of households', 'Number of households in fuel poverty',
       'Proportion of households fuel poor (%)'],
      dtype='object')

In [ ]:
# Limit to Lewisham LSOAs
lbl_fp = d[d.LSOA11CD.isin(lbl_lsoa_list)].copy()

In [ ]:
# Add Ward codes
lsoa_wd22_map = lsoa_wd22_join.set_index('LSOA11CD').WD22CD_proposed.to_dict()
lbl_fp['WD22CD_proposed'] = lbl_fp.LSOA11CD.map(lambda lsoa: lsoa_wd22_map[lsoa])

In [ ]:
# Export
lbl_fp.to_csv(f"{out_dir}/lbl_fp_lsoa11.csv", index=False)
IPython.display.display(len(lbl_fp))
IPython.display.display(lbl_fp.head())

169

,LSOA11CD,LSOA11NM,LAD19CD,LAD19NM,Region,Number of households,Number of households in fuel poverty,Proportion of households fuel poor (%),WD22CD_proposed
3121,E01003189,Lewisham 034A,E09000023,Lewisham,London,667,179,26.836582,E05013714
3122,E01003190,Lewisham 034B,E09000023,Lewisham,London,861,136,15.795587,E05013714
3123,E01003191,Lewisham 034C,E09000023,Lewisham,London,710,161,22.676056,E05013714
3124,E01003192,Lewisham 034D,E09000023,Lewisham,London,792,160,20.202020,E05013714
3125,E01003193,Lewisham 029A,E09000023,Lewisham,London,711,112,15.752461,E05013729


In [ ]:
lbl_fp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169 entries, 3121 to 32452
Data columns (total 9 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   LSOA11CD                                169 non-null    object 
 1   LSOA11NM                                169 non-null    object 
 2   LAD19CD                                 169 non-null    object 
 3   LAD19NM                                 169 non-null    object 
 4   Region                                  169 non-null    object 
 5   Number of households                    169 non-null    int64  
 6   Number of households in fuel poverty    169 non-null    int64  
 7   Proportion of households fuel poor (%)  169 non-null    float64
 8   WD22CD_proposed                         169 non-null    object 
dtypes: float64(1), int64(2), object(6)
memory usage: 13.2+ KB
